### 일반

In [1]:
import numpy as np
import pandas as pd

# 데이터 개수 설정
n_samples = 10000

# 외생 변수 Z 생성 (latent, 최종 데이터에는 포함되지 않음)
np.random.seed(42)

Z = np.random.uniform(-1, 1, n_samples)

# X1, X2, X3 생성 (Z에 선형 관계와 노이즈 추가)
X1 = 0.5 * Z + np.random.normal(0, 0.1, n_samples)
X2 = -0.7 * Z + np.random.normal(0, 0.1, n_samples)
X3 = 1.2 * Z + np.random.normal(0, 0.1, n_samples)

# T1, T2 생성 (Z 및 T1에 선형 관계와 노이즈 추가)
T1 = 0.3 * Z + np.random.normal(0, 0.1, n_samples)
T2 = 0.8 * T1 + 0.5 * Z + np.random.normal(0, 0.1, n_samples)

# Y 생성 (Z, T2에 선형 관계와 노이즈 추가)
Y = 1.5 * Z + 2.0 * T2 + np.random.normal(0, 0.2, n_samples)

# 데이터 프레임 생성 (Z는 포함시키지 않음)
data = pd.DataFrame({
    'X1': X1,
    'X2': X2,
    'X3': X3,
    'T1': T1,
    'T2': T2,
    'Y': Y
})

# 데이터 확인
print(data.head())
data.to_csv('synthetic.csv', index=False)


         X1        X2        X3        T1        T2         Y
0 -0.274246  0.114029 -0.222736 -0.197550 -0.141564 -0.482924
1  0.338196 -0.670267  1.082468  0.208837  0.687798  2.986315
2  0.270876 -0.427974  0.622267 -0.047693  0.143983  0.819783
3 -0.018729 -0.227047  0.135349  0.051812  0.280557  0.716724
4 -0.232718  0.525012 -0.811217 -0.147677 -0.416117 -1.942434


### 시계열

In [3]:
import numpy as np
import pandas as pd

n_series = 1000  # 시계열 데이터 개수
n_length = 5     # 각 시계열의 길이

# 파라미터 설정
phi = 0.9
# np.random.seed(42)

# 시계열 데이터 구조 초기화
Z = np.zeros((n_series, n_length))
X1 = np.zeros((n_series, n_length))
X2 = np.zeros((n_series, n_length))
X3 = np.zeros((n_series, n_length))
T1 = np.zeros((n_series, n_length))
T2 = np.zeros((n_series, n_length))
Y = np.zeros((n_series, n_length))

for i in range(n_series):
    Z[i, 0] = np.random.exponential(1)  # 초기값을 양수로 설정 (지수 분포)
    for t in range(1, n_length):
        Z[i, t] = phi * Z[i, t-1] + np.random.exponential(0.1)  # 양수로 제한


# 나머지 변수 계산
for i in range(n_series):
    for t in range(n_length):
        X1[i, t] = 0.5 * Z[i, t] + np.random.normal(0, 0.1)
        X2[i, t] = -0.7 * Z[i, t] + np.random.normal(0, 0.1)
        X3[i, t] = 1.2 * Z[i, t] + np.random.normal(0, 0.1)
        T1[i, t] = 0.3 * Z[i, t] + np.random.normal(0, 0.1)
        T2[i, t] = 0.8 * T1[i, t] + 0.3 * Z[i, t] + np.random.normal(0, 0.1)
        # Y[i, t]를 시간에 따라 감소하는 선형 트렌드를 추가하여 계산
        decay_factor = 1.0 - 0.2 * t  # 감소 계수, 시간에 따라 감소
        Y[i, t] = (0.7 * Z[i, t] - 1.3 * T2[i, t] + np.random.normal(0, 0.2)) #* decay_factor


# 데이터 프레임으로 변환
data_frames = [pd.DataFrame({
    'Z': Z[i, :],
    'X1': X1[i, :],
    'X2': X2[i, :],
    'X3': X3[i, :],
    'T1': T1[i, :],
    'T2': T2[i, :],
    'Y': Y[i, :]
}) for i in range(n_series)]

# 첫 번째 시계열 데이터 확인
print(data_frames[0])


          Z        X1        X2        X3        T1        T2         Y
0  0.516732  0.267814 -0.272633  0.578618  0.197903  0.458707  0.129085
1  0.469930  0.201121 -0.339048  0.739961  0.140373  0.301187 -0.446921
2  0.462342  0.210427 -0.241025  0.659842  0.396955  0.454402 -0.301617
3  0.430938  0.032348 -0.245854  0.702234  0.227834  0.370832 -0.226085
4  0.395424  0.327843 -0.109247  0.322533  0.209899  0.279360 -0.096919


### 더 우리와 같은 setting (T1, T2 가 seq별로 고정)

In [122]:
import numpy as np
import pandas as pd
import pickle

n_series = 1000  # 시계열 데이터 개수
n_length = 5     # 각 시계열의 길이

# 파라미터 설정
phi = 0.9

# 시계열 데이터 구조 초기화
Z = np.zeros((n_series, n_length))
X1 = np.zeros((n_series, n_length))
X2 = np.zeros((n_series, n_length))
X3 = np.zeros((n_series, n_length))
X4 = np.zeros((n_series, n_length))
T1 = np.zeros((n_series, n_length))
T2 = np.zeros((n_series, n_length))
P = np.zeros((n_series, n_length))
Y = np.zeros((n_series, n_length))

for i in range(n_series):
    Z[i, 0] = np.random.exponential(1)  # 초기값을 양수로 설정 (지수 분포)
    for t in range(1, n_length):
        Z[i, t] = phi * Z[i, t-1] + np.random.exponential(0.05)  # 양수로 제한

for i in range(n_series):
    # T1과 T2 계산을 한 번만 수행하고 모든 t에 대해 같은 값을 사용
    T1[i, :] = 0.3 * Z[i, 0] + np.random.normal(0, 0.05)  # T1 초기화
    T2[i, :] = 0.8 * T1[i, 0] + 0.3 * Z[i, 0] + np.random.normal(0, 0.05)  # T2 초기화
    
    for t in range(n_length):
        X1[i, t] = 0.5 * Z[i, t] + np.random.normal(0, 0.05)
        X2[i, t] = -0.7 * Z[i, t] + np.random.normal(0, 0.05)
        X3[i, t] = 1.2 * Z[i, t] + np.random.normal(0, 0.05)
        X4[i, t] = 0.2 * Z[i, t] + np.random.normal(0, 0.05) + 1
        # Y[i, t]를 시간에 따라 감소하는 선형 트렌드를 추가하여 계산
        # decay_factor = 1.0 - 0.2 * t  # 감소 계수, 시간에 따라 감소
        P[i, t] = max(0, (0.7 * Z[i, t] - 1.3 * T2[i, 0] ) + 1) #* decay_factor  # T2[i, 0] 사용

for i in range(n_series):
    for j in range(n_length):
        Y[i, j] = np.sum(P[i, j:n_length])  # j부터 n_length까지의 합

# 데이터 프레임으로 변환
data_frames = [pd.DataFrame({
    'Z': Z[i, :],
    'X1': X1[i, :],
    'X2': X2[i, :],
    'X3': X3[i, :],
    'X4': X4[i, :],
    'T1': T1[i, :],
    'T2': T2[i, :],
    'P': P[i, :],
    'Y': Y[i, :]
}) for i in range(n_series)]

# 첫 번째 시계열 데이터 확인
print(data_frames[0])

with open('synthetic_ts.pkl', 'wb') as f:
    pickle.dump(data_frames, f)


          Z        X1        X2        X3        X4        T1        T2  \
0  0.233453  0.129841 -0.163832  0.293045  0.972370  0.019833  0.090322   
1  0.292146  0.150837 -0.346751  0.298301  1.072382  0.019833  0.090322   
2  0.277541  0.101334 -0.231111  0.351270  1.045465  0.019833  0.090322   
3  0.391738  0.282074 -0.328807  0.501929  1.134855  0.019833  0.090322   
4  0.353439  0.137889 -0.266856  0.418021  1.098971  0.019833  0.090322   

          P         Y  
0  1.045998  5.496728  
1  1.087084  4.450730  
2  1.076860  3.363646  
3  1.156798  2.286786  
4  1.129988  1.129988  


In [123]:
import torch
from torch.utils.data import Dataset, DataLoader
class TimeSeriesDataset(Dataset):
    def __init__(self, data_frames):
        # Convert each dataframe column into a numpy array and then into a tensor
        self.X1 = torch.tensor(np.array([df['X1'].values for df in data_frames]), dtype=torch.float32)
        self.X2 = torch.tensor(np.array([df['X2'].values for df in data_frames]), dtype=torch.float32)
        self.X3 = torch.tensor(np.array([df['X3'].values for df in data_frames]), dtype=torch.float32)
        self.X4 = torch.tensor(np.array([df['X4'].values for df in data_frames]), dtype=torch.float32)
        self.T1 = torch.tensor(np.array([df['T1'].values for df in data_frames]), dtype=torch.float32)
        self.T2 = torch.tensor(np.array([df['T2'].values for df in data_frames]), dtype=torch.float32)
        self.P = torch.tensor(np.array([df['P'].values for df in data_frames]), dtype=torch.float32)
        self.Y = torch.tensor(np.array([df['Y'].values for df in data_frames]), dtype=torch.float32)

    def __len__(self):
        # Return the total length of the dataset
        return len(self.Y)
    
    def __getitem__(self, idx):
        # Return the tensors at the specific index
        x1 = self.X1[idx]
        x2 = self.X2[idx]
        x3 = self.X3[idx]
        x4 = self.X4[idx]
        y = torch.sum(self.P[idx])
        t1 = self.T1[idx]
        t2 = self.T2[idx]
        t = torch.cat((t1.unsqueeze(0), t2.unsqueeze(0)), dim=0)
        index_tensor = torch.tensor([0, 1, 2, 3, 4], dtype=torch.float)
        x1_length = x1.shape[0]

        return x1, x2, x3, x4, x1_length, y, index_tensor, torch.mean(t, dim=-1)

# Load the dataset from a CSV file and convert it back to individual dataframes
with open('synthetic_ts.pkl', 'rb') as f:
    data_frames = pickle.load(f)
    
dataset = TimeSeriesDataset(data_frames)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# 배치 데이터 처리 예시
for x1, x2, x3, x4, x1_length, y, index_tensor, t in dataloader:
    # print(x1, x2, x3, x4, x1_length, y, index_tensor, t)
    print(x1.shape)
    break

torch.Size([10, 5])


In [3]:
import dowhy
from dowhy import CausalModel
import numpy as np
import pandas as pd
import networkx as nx
import os
import graphviz
from causallearn.search.FCMBased.lingam.utils import make_dot
from causallearn.search.FCMBased import lingam
from causallearn.search.ConstraintBased.PC import pc
from causallearn.search.ScoreBased.GES import ges
from causallearn.search.HiddenCausal.GIN.GIN import GIN
from causallearn.utils.GraphUtils import GraphUtils
import dowhy.datasets
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import io

np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)



def make_graph(adjacency_matrix, labels=None):
    idx = np.abs(adjacency_matrix) > 0.01
    dirs = np.where(idx)
    d = graphviz.Digraph(engine='dot')
    names = labels if labels else [f'x{i}' for i in range(len(adjacency_matrix))]
    for name in names:
        d.node(name)
    for to, from_, coef in zip(dirs[0], dirs[1], adjacency_matrix[idx]):
        d.edge(names[from_], names[to], label=str(coef))
    return d

def str_to_dot(string):
    '''
    Converts input string from graphviz library to valid DOT graph format.
    '''
    graph = string.strip().replace('\n', ';').replace('\t','')
    graph = graph[:9] + graph[10:-2] + graph[-1] # Removing unnecessary characters from string
    return graph

for tag in ['']:
    data_path = f'synthetic.csv'
    data = pd.read_csv(data_path)
    # data = data.drop(['cut_date','diff_days'], axis=1)
    labels = [f'{col}' for i, col in enumerate(data.columns)]
    tag=f'{tag}_synthtetic'
        
    if not os.path.exists(f'./fig/pc{tag}.png'):
        print("discovering with pc algorithm")
        data = data.to_numpy()

        cg = pc(data)

        # Visualization using pydot
        pyd = GraphUtils.to_pydot(cg.G, labels=labels)
        tmp_png = pyd.create_png(f="png")
        fp = io.BytesIO(tmp_png)
        img = mpimg.imread(fp, format='png')
        plt.axis('off')
        plt.imshow(img)
        plt.savefig(f'./fig/pc{tag}.png', dpi=300)


    if not os.path.exists(f'./fig/ges{tag}.png'):
        print("discovering with ges algorithm")
        # default parameters
        Record = ges(data)

        # Visualization using pydot
        pyd = GraphUtils.to_pydot(Record['G'], labels=labels)
        tmp_png = pyd.create_png(f="png")
        fp = io.BytesIO(tmp_png)
        img = mpimg.imread(fp, format='png')
        plt.axis('off')
        plt.imshow(img)
        plt.savefig(f'./fig/ges{tag}.png', dpi=300)


    if not os.path.exists(f'./fig/lingam{tag}.png'):
        print("discovering with lingam algorithm")
        model = lingam.ICALiNGAM()
        model.fit(data)

        
        dot_object = make_dot(model.adjacency_matrix_, labels=labels)


        # 'dot_object'는 'Digraph' 객체입니다.
        dot_str = dot_object.source  # DOT 언어로 변환

        # DOT 문자열을 사용하여 Source 객체 생성
        dot_source = graphviz.Source(dot_str)

        # 이미지 파일로 렌더링하여 저장
        dot_source.render(f'./fig/lingam{tag}', format='png', engine='dot')


/data1/bubble3jh/anaconda3/envs/cet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
